In [1]:
import numpy as np

my_list = [1,2,3,4,5]
my_array=np.array(my_list)
print(my_list)
print(my_array)
print("="*10)
print(type(my_list))
print(type(my_array))

[1, 2, 3, 4, 5]
[1 2 3 4 5]
<class 'list'>
<class 'numpy.ndarray'>


In [40]:
mystring= "hello mohamed ali"
print(mystring.find('m'))
print(mystring.strip().split(","))
print(mystring.capitalize().strip())

fruits = ["apple","banana","orange"]
for i,v in enumerate(fruits):
    print(v)
    
for i in range(1,6):
    if i==3:
        continue
    print(i)
print('#'*50)

fakeList=['hi','hello','mohamed']
fakeList.extend(['Ali','Ahmed'])
print(fakeList)
print('#'*50)

clone = fakeList
print(clone)

6
['hello mohamed ali']
Hello mohamed ali
apple
banana
orange
1
2
4
5
##################################################
['hi', 'hello', 'mohamed', 'Ali', 'Ahmed']
##################################################
['hi', 'hello', 'mohamed', 'Ali', 'Ahmed']


In [19]:
import json
import requests
import pandas as pd
import datetime as dt

# Step 1: Extract
def get_data():
    url = 'https://www.freepublicapis.com/api/random'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data
    return None

# Step 2: Transform
def transdata():
    data = get_data()
    if data:
        df = pd.DataFrame([data]) if isinstance(data, dict) else pd.DataFrame(data)
        return df
    else:
        print("No data returned.")
        return pd.DataFrame()

# Step 3: Run the ETL steps
print("Getting data...")
raw_data = get_data()
print("Finished getting data.")

print("\nTransforming data...")
df = transdata()
print("Finished transforming data.")

# Optional: display or save the DataFrame
print(df.head())


Getting data...
Finished getting data.

Transforming data...
Finished transforming data.
    id  emoji                  title  \
0  254  🕵️‍♂️  User Agent Lookup API   

                                         description  \
0  This API allows users to parse and retrieve in...   

                      documentation  methods  health  \
0  https://www.useragentlookup.com/        1     100   

                                             source  
0  https://freepublicapis.com/user-agent-lookup-api  


In [25]:
from bs4 import BeautifulSoup # type: ignore
import requests

source = requests.get('https://www.democomp.com/').text
soup = BeautifulSoup(source , 'html.parser')
print(soup.prettify())
artisits = soup.find_all('a')

for artisit in artisits:
    names = artisit.content[0]
    fulllink = artisit.get('href')
    print(names)
    print(fulllink)

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://www.democomp.com/xmlrpc.php" rel="pingback"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="icon" sizes="96x96" type="image/png"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="icon" sizes="160x160" type="image/png"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="icon" sizes="192x192" type="image/png"/>
  <link href="https://www.democomp.com/wp-content/uploads/2018/11/fav.png" rel="apple-touch-icon" sizes

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = 'Movies.db'
table_name = 'Top_50'
csv_path = 'top_50_films.csv'
df = pd.DataFrame(columns=["Average Rank","Film","Year"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count < 50 :
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": int(col[0].contents[0]),
                         "Film": str(col[1].contents[0]),
                         "Year": int(col[2].contents[0])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break

print(df)

df.to_csv(csv_path)

conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'


def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            if col[0].find('a') is not None:
                data_dict = {
                    "Country": col[0].a.contents[0],
                    "GDP_USD_millions": col[2].contents[0]
                }
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df, df1], ignore_index=True)
    return df

df = extract(url, table_attribs)
df.to_csv(csv_path)